In [1]:
import pandas as pd
import numpy as np

In [2]:
# get raw data
df = pd.read_csv("raw_data/Machine Learning Answers.md", sep="\n")

In [3]:
df["## Machine Learning"].str[0:5].value_counts()

- [ ]    287
####      96
- [x]     96
[Refe      7
**Exp      7
![Mac      6
[Sour      3
![ima      3
  **E      1
  `Un      1
`The       1
`that      1
`// t      1
  `//      1
`//Yo      1
Name: ## Machine Learning, dtype: int64

In [4]:
# clean df of explanations, references for now
df = df[df["## Machine Learning"].str[0:5].isin(["- [ ]",
                                                 "#### ",
                                                 "- [x]",
                                                 "![Mac",
                                                 "![ima"])]

In [5]:
# initialize results DataFrame
res = pd.DataFrame(columns=["question", "answ1", "answ2", "answ3", "answ4", "correct"],
                    index=list(range(len(df) // 5)))

In [110]:
# order question and answers
for i in range(len(df)):
    value = df.iloc[i,0]
    res.iloc[i // 5, i % 5] = value
    if value[0:5] == "- [x]":
        res.iloc[i // 5, 5] = i % 5
    if value[0] == "#":
        i += 1

In [ ]:
# new order including images
ans_list = []
for i in range(len(df)):
    value = df.iloc[i,0]
    res_tmp = pd.DataFrame(columns=["question", "answ1", "answ2", "answ3", "answ4", "correct", "image"], index=1)
    if value[0] == "#":
        res_tmp["question"] = value
        ans_list = []
    if value[0:5] == "- [ ]":
        ans_list.append(value)
    if value[0:5] == "- [ ]":
        ans_list.append(value)
    

In [6]:
# turn df into list
# loop through res, take elements from list (pop)
lst = list(df["## Machine Learning"])
res_lst = []
while len(lst) > 0:
    res_tmp = pd.DataFrame(columns=["question", "answ1", "answ2", "answ3", "answ4", "correct", "image"], index=[0])
    ans_lst = []
    if lst[0][0] == "#":
        res_tmp["question"] = lst.pop(0)
    if lst[0][0] == "-":
        ans_lst.append(lst.pop(0))

['a', 'c']

In [121]:
# Clean answers
res["answ1"] = res["answ1"].str[5:]
res["answ2"] = res["answ2"].str[5:]
res["answ3"] = res["answ3"].str[5:]
res["answ4"] = res["answ4"].str[5:]

In [124]:
# Clean question
res["question"] = res["question"].str.lstrip("#### ")

In [125]:
res.tail(2)

,question,answ1,answ2,answ3,answ4,correct
94,Q95. When is a decision tree most commonly used?,with big data products,for supervised machine learning binary classi...,to find thd best data cluster,"to determine ""Q"" in Q-learning reinforcement ...",2
95,Q96. An organisation that owns dozens of shopp...,most machine learning models are not designed...,it might be unethical for the business to ide...,it will be difficult to decide between superv...,the image in the video would not be high qual...,2


In [126]:
# save csv
res.to_csv("data/results.csv", index=False)